In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt


from scipy.signal import savgol_filter as sf

In [22]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,159,168,169,177,178,179,180,181,182,183,186,187,190,192]

#Import route shapefile
route = '../data/rt' + str(rt_list[0]) + '_pts2.shp'


In [35]:
def create_gdf(shapefile, pt_distance):
    """
        Loads shapefile into GeoDataFrame with original point distance

        Parameters
        ----------
        shapefile: route geospatial data (.shp file)
        pt_distance: distance between points (in ft)

        Returns
        -------
        route_shp: route_df
        """

    route_df = gpd.read_file(shapefile)
    
    x = []

    for i in route_df.index:
    
        x.append(i*pt_distance)
    

    route_df['distance'] = x
    route_df = route_df.rename(columns = {'Z':'elevation'})
    route_df = route_df.rename(columns = {'SPEED_LIM':'speed_limit'})
    #route_df = route_df.drop(columns=['Id','ORIG_FID', 'SHAPE_Leng'])
    route_df = route_df[['geometry', 'distance', 'elevation', 'speed_limit']]
    
    return route_df

In [36]:
route_df = create_gdf(route, 6)
route_df

,geometry,distance,elevation,speed_limit
0,POINT (-122.38813 47.56295),0,324.449184,25
1,POINT (-122.38810 47.56295),6,324.591054,25
2,POINT (-122.38808 47.56295),12,324.689024,25
3,POINT (-122.38806 47.56295),18,324.780330,25
4,POINT (-122.38803 47.56295),24,324.841638,25
...,...,...,...,...
9714,POINT (-122.38813 47.56285),58284,323.775553,25
9715,POINT (-122.38813 47.56288),58290,347.445554,25
9716,POINT (-122.38813 47.56290),58296,354.685970,25
9717,POINT (-122.38813 47.56292),58302,324.268711,25


In [37]:
def metric_df(df, frequency):
    """
        Converts dataframe units to metric. Also, set how many points to evaluate. 

        Parameters
        ----------
        df: geodataframe
        frequency: how many points to keep. For example, if frequency=6, every 6th point is kept. 

        Returns
        -------
        route_shp: metric route dataframe
        """
    df = df.iloc[::frequency]
    df = df.reset_index()
    del df['index']
    df['distance'] *= 0.3048
    df['elevation'] *= 0.3048
    df['speed_limit']/= 2.237
    
    return df, df['distance'][1]

In [38]:
route_df, distance = metric_df(route_df, 6)
route_df.head()

,geometry,distance,elevation,speed_limit
0,POINT (-122.38813 47.56295),0.0000,98.892111,11.175682
1,POINT (-122.38798 47.56295),10.9728,99.063447,11.175682
2,POINT (-122.38784 47.56295),21.9456,100.796006,11.175682
3,POINT (-122.38769 47.56295),32.9184,99.497336,11.175682
4,POINT (-122.38754 47.56295),43.8912,99.715647,11.175682


In [39]:
route_df

,geometry,distance,elevation,speed_limit
0,POINT (-122.38813 47.56295),0.0000,98.892111,11.175682
1,POINT (-122.38798 47.56295),10.9728,99.063447,11.175682
2,POINT (-122.38784 47.56295),21.9456,100.796006,11.175682
3,POINT (-122.38769 47.56295),32.9184,99.497336,11.175682
4,POINT (-122.38754 47.56295),43.8912,99.715647,11.175682
...,...,...,...,...
1615,POINT (-122.38813 47.56227),17721.0720,97.261303,11.175682
1616,POINT (-122.38813 47.56241),17732.0448,97.616852,11.175682
1617,POINT (-122.38813 47.56256),17743.0176,97.986252,11.175682
1618,POINT (-122.38813 47.56271),17753.9904,98.358077,11.175682


In [40]:
route_df['elevation'].values

array([ 98.89211128,  99.06344719, 100.79600628, ...,  97.98625205,
        98.35807727,  98.68678851])

In [41]:
def filters(route_df):
    
    points = route_df['elevation'].values

    y_new = sf(points, 43, 3,  axis = 0) 
    
    return y_new

In [42]:
y_new = filters(route_df)
y_new

array([ 99.39808966,  99.87477354, 100.27235379, ...,  98.0396214 ,
        98.57525306,  99.15372437])

In [43]:
def grade(y_new, distance):
    
    grade_SG = [0]

    for i in range(1,len(y_new)):
        grade_SG.append((y_new[i]-y_new[i-1])/distance)
    
    return grade_SG

In [44]:
grade_SG = grade(y_new, distance)

In [45]:
def wrapper(shapefile, pt_distance, frequency):

    route_df = create_gdf(shapefile, pt_distance)
    
    route_df, distance = metric_df(route_df, frequency)

    route_df['elevation'] = filters(route_df)

    route_df['grade'] = grade(route_df['elevation'].values, distance)

    return route_df

In [46]:
wrapper(route, 6, 6)

,geometry,distance,elevation,speed_limit,grade
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889
...,...,...,...,...,...
1615,POINT (-122.38813 47.56227),17721.0720,97.091160,11.175682,0.037883
1616,POINT (-122.38813 47.56241),17732.0448,97.545400,11.175682,0.041397
1617,POINT (-122.38813 47.56256),17743.0176,98.039621,11.175682,0.045041
1618,POINT (-122.38813 47.56271),17753.9904,98.575253,11.175682,0.048814
